In [1]:
from glob import glob
import json
import random
from tqdm import tqdm

In [2]:
ls = []
with open('ls.jsonl') as fopen:
    for l in fopen:
        ls.append(json.loads(l))
        
len(ls)

213954

In [3]:
glob('/home/husein/ssd3/translation/mixtral-summary.texts*.splitted.requested')

['/home/husein/ssd3/translation/mixtral-summary.texts09.splitted.requested',
 '/home/husein/ssd3/translation/mixtral-summary.texts00.splitted.requested',
 '/home/husein/ssd3/translation/mixtral-summary.texts04.splitted.requested',
 '/home/husein/ssd3/translation/mixtral-summary.texts02.splitted.requested',
 '/home/husein/ssd3/translation/mixtral-summary.texts08.splitted.requested',
 '/home/husein/ssd3/translation/mixtral-summary.texts06.splitted.requested',
 '/home/husein/ssd3/translation/mixtral-summary.texts01.splitted.requested',
 '/home/husein/ssd3/translation/mixtral-summary.texts05.splitted.requested',
 '/home/husein/ssd3/translation/mixtral-summary.texts03.splitted.requested',
 '/home/husein/ssd3/translation/mixtral-summary.texts07.splitted.requested']

In [5]:
import re

mapping = {}
for f in glob('/home/husein/ssd3/translation/mixtral-summary.texts*.splitted.requested'):
    with open(f) as fopen:
        for l in fopen:
            l = json.loads(l)
            if 'Source text\nclear\nLook up details' in l['r']['result']:
                continue
            if 'clear\nLook up details' in l['r']['result']:
                continue
            if l['r']['result'].startswith('Source text\n'):
                continue

            n = l['r']['result']
            hypens = re.findall('\w+ -\w+', n)
            for h in hypens:
                splitted = h.split('-')
                if len(splitted) != 2:
                    continue
                splitted = [s.strip() for s in splitted]
                splitted = '-'.join(splitted)
                n = n.replace(h, splitted)
            mapping[l['src']] = n

len(mapping)

191676

In [11]:
import os

with open('mixtral-malaysian-abstractive-summarization.jsonl', 'w') as fopen_l:
    for no, l in enumerate(ls):
        filename = f'summary/{no}.json'
        if not os.path.exists(filename):
            continue
        
        with open(filename) as fopen:
            t = json.load(fopen).strip()
            
        d = {
            'source': l[-1],
            'text': l[0],
            'summary': t,
            'summary_ms': mapping.get(t)
        }
        
        fopen_l.write(f'{json.dumps(d)}\n')

In [12]:
!wc -l mixtral-malaysian-abstractive-summarization.jsonl

195594 mixtral-malaysian-abstractive-summarization.jsonl


In [13]:
!head -n 1 mixtral-malaysian-abstractive-summarization.jsonl

{"source": "gamerbraves.com.jsonl", "text": "Attack on Titan Brave Order has opened for Pre Registration\n\n\nThe new mobile game Attack on Titan Brave Order from developer Enish has opened for pre-registration\n\n\n\nHere\u2019s the trailer:\n\n\n\nRelated Posts   Pokemon Sleep Now Available In Malaysia   GamerBraves Newsletter Vol. 103 \u2013 The Glory of The Barbie/Oppenheimer Double Feature   According to 4Gamer, the first promotional movie has been released, and the pre-registration campaign has started, in which the number of \u2018red brilliant\u2019, the currency used for gacha will increase according to the number of registered users.\nHere are the rates:\nFollow us on Twitter\nFollow @GamerBraves\nand Tweet us\nTweet to @GamerBraves\n\n\nOver 30,000 registrations: 2 gacha worth of red brilliant\nOver 50,000 users: 4 gacha worth of red brilliant\nMore than 100,000 players will receive 6 gacha worth of red stones.\nMore than 200,000 users: 8 times the number of gachas\nOver 300

In [14]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj='mixtral-malaysian-abstractive-summarization.jsonl',
    path_in_repo='mixtral-malaysian-abstractive-summarization.jsonl',
    repo_id='mesolitica/mixtral-malaysian-abstractive-summarization',
    repo_type='dataset',
)

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/mixtral-malaysian-abstractive-summarization/commit/f2aacb31fb0e1d231e05b37a529812ff8932ca7e', commit_message='Upload mixtral-malaysian-abstractive-summarization.jsonl with huggingface_hub', commit_description='', oid='f2aacb31fb0e1d231e05b37a529812ff8932ca7e', pr_url=None, pr_revision=None, pr_num=None)